Bharat Nandan Bodala.  github- @mlgfamlmao


PROBLEM CASE -  predict price value of a car based on a set of features.

features- {
dateCrawled: Date the car was crawled. (Date)
name: Name of the car. (String)
seller: Type of seller (private or dealer). (String)
offerType: Type of offer (e.g. sale, repair, etc.). (String)
price: Price of the car. (Integer)
abtest: Test type (A or B). (String)
vehicleType: Type of vehicle (e.g. SUV, sedan, etc.). (String)
yearOfRegistration: Year the car was registered. (Integer)
gearbox: Type of gearbox (manual or automatic). (String)
powerPS: Power of the car in PS. (Integer)
model: Model of the car. (String)
kilometer: Kilometers the car has been driven. (Integer)
monthOfRegistration: Month the car was registered. (Integer)
fuelType: Type of fuel (e.g. diesel, petrol, etc.). (String)
brand: Brand of the car. (String)
notRepairedDamage: Whether or not the car has any damage that has not been repaired. (String)
dateCreated: Date the car was created. (Date)
nrOfPictures: Number of pictures of the car. (Integer)
postalCode: Postal code of the car. (Integer)
lastSeen: Date the car was last seen. (Date)
}

Approach:
        -My initial approach was to clean up data (dropped some useless columns such as index etc and removed outlier values.) , and then, I generated correlation matrices and pyplots. 
        -Many of the features were deemed useless by observation except powerPS.
        -Before proceeding to build a model on one feature, I tried to perform 3 bit encoding to vehicleType and binary encoding to gearbox, but again, the data is not good and no type of meaningful polynomial relation is to be found. 
        -Either ways, I have decided to train a model based on one feature powerPS.
        -Initial chosen hyperparameters: epochs= 50, learning_rate= 0.001, batch_size= 500.
        -Used a 1 density keras model with gradient descent approach. Used mean squared error loss as convergence metric.
        -Since aforementioned keras model has some unexpected bugs, scikit SGD regressor was used. It provided poor results

Conclusion:
        Poor data. 

In [ ]:
import pandas as pd
import tensorflow
import numpy as np
import keras
import seaborn as sns
import kagglehub
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor
import joblib

In [ ]:
dataset= pd.read_csv("autos.csv")
dataset["gearbox"]= dataset["gearbox"].apply(lambda x : 1 if x=="automatik" else 0)
cleaned = dataset.drop(['index'], axis=1)
print(cleaned.head(200))
print("--")
cleaned["price"]= cleaned["price"]/100000
print(cleaned['vehicleType'].unique().tolist())

In [ ]:
sns.stripplot(x='vehicleType', y='price', data=cleaned)
#too much similarity between vehicleType and prices


In [ ]:
sns.stripplot(x='model', y = 'price', data = cleaned)
#too much similarity between model types and prices

In [ ]:
def vehicleEncoding():
    #Assign each vehicleType to a 3 bit binary number (000,001 etc)
    #Idea scrapped as doing this may not be beneficial at all based on aforementioned reasons.
    return


In [ ]:
print(cleaned.describe())
cleaned.corr(numeric_only= True)

In [ ]:
X = cleaned.drop(["price"],axis=1)
y= cleaned["price"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
print(y_train)

In [ ]:
sns.pairplot(cleaned.head(2000),x_vars=["price","powerPS","gearbox"],y_vars=["price","powerPS","gearbox"])

In [ ]:
#This method is bringing some errors which i cannot debug; used scikit insetad (next code block).
def build_model(learning_rate,num_features):
  inputs= keras.Input(shape = (num_features,))
  output = keras.layers.Dense(units= 1)(inputs)
  model = keras.Model(inputs= inputs, outputs= output)

  #compile the model
  print("Model metrics:", model.metrics_names)

  model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
                loss="mean_squared_error",
                metrics=[keras.metrics.RootMeanSquaredError()])

  return model

def train_model(model, features, label, epochs, batch_size):



  history = model.fit(x=features,
                      y=label,
                      batch_size=batch_size,
                      epochs=epochs)


  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  epochs = history.epoch



  hist = pd.DataFrame(history.history)


  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse



In [ ]:
learning_rate= 0.001
batch_size= 50
epochs= 50
features= ['powerPS']
feature_values = X_train.loc[:, features] 
label = "price"
#model = build_model(learning_rate, len(features)) cannot debug??
model = SGDRegressor(alpha=0.01, learning_rate='constant', eta0=learning_rate,
                     loss='epsilon_insensitive', max_iter=epochs, n_iter_no_change=20, penalty='l2')
model.fit(X_train[features], y_train)

trained_weight = model.coef_
trained_bias = model.intercept_
trained_epochs = model.n_iter_
# rmse calculation is not done yet, you need to compute it manually
from sklearn.metrics import mean_squared_error
import numpy as np
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train[features])))

print("Weights:", trained_weight)
print("Bias:", trained_bias)
print("Epochs:", trained_epochs)
print("Final RMSE:", rmse)

joblib.dump(model, 'FinalUsedCarsModel.pkl')


#Not going to test on test data. Credit to this decision goes to the poor RMSE value.
#Conclusion: poor data.

